In [14]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.types import Command,interrupt
from langgraph.checkpoint.memory import MemorySaver

In [15]:
memory = MemorySaver()

class State(TypedDict):
    text:str

def node_a(state: State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "text":state["text"] + "A"
        }
    )
def node_b(state: State):
    print("Node B")
    human_response = interrupt("Node B: Do you want to go to node C or D?")
    print("Human Response",human_response)
    if human_response == "C":
        return Command(
            goto="node_c",
            update={
                "text":state["text"] + "B"
            }
        )
    elif human_response == "D":
        return Command(
            goto="node_d",
            update={
                "text":state["text"] + "B"
            }
        )
    else:
        print("Invalid response, going to node C by default")
  
def node_C(state: State) -> State:
    print("Node C")
    return Command(
        goto=END,
        update={
            "text":state["text"] + "C"
        }
    )
def node_D(state: State) -> State:
    print("Node D")
    return Command(
        goto=END,
        update={
            "text":state["text"] + "D"
        }
    )

graph = StateGraph(State)
graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_C)
graph.add_node("node_d", node_D)
graph.set_entry_point("node_a")

app= graph.compile(checkpointer=memory)

config={"configurable":{"thread_id":"1"}}

response =app.invoke({"text":""},config=config,stream_mode="updates")

response

    

Node A
Node B


[{'node_a': {'text': 'A'}},
 {'__interrupt__': (Interrupt(value='Node B: Do you want to go to node C or D?', resumable=True, ns=['node_b:6fc7a809-591b-bf89-5201-32ad6f7c7b88']),)}]

In [16]:
print(app.get_state(config).next)

('node_b',)


In [21]:
second_result = app.invoke(Command(resume="D"),config=config,stream_mode="updates")
second_result

[]